In [1]:
# move to visualcode and make it efficient

from sklearn.model_selection import train_test_split
import torch, os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics.functional as metrics
import numpy as np
import pandas as pd

%matplotlib inline

C:\Users\nchau\anaconda3\envs\env_pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
data_path = './data/processed_data.csv'
data = pd.read_csv(data_path)

X_train = data.iloc[:, 0]
Y_train = data.iloc[:, 1:]

In [15]:
img = X_train.iloc[3].as_matrix()

AttributeError: 'str' object has no attribute 'as_matrix'

In [14]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target, reduction='sum')
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            accuracy = metrics.accuracy(output, target)

            print(
                f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100 * batch_idx / len(train_loader):.0f}%)]'
                f'\tLoss: {loss.detach().item():.6f}'
                f'\tAccuracy: {accuracy.detach().item():.2f}'
            )
            
@torch.no_grad()
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)

        # get model output
        output = model(data)

        # calculate loss
        test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss

        # get most likely class label
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability

        # count the number of correct predictions
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100 * correct / len(test_loader.dataset)))

In [15]:
use_cuda = torch.cuda.is_available()
print(f"CUDA is {'' if use_cuda else 'not '}available")
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:
    torch.cuda.set_per_process_memory_fraction(0.22)

CUDA is not available


In [16]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        # 4 convolution layers, with a non-linear activation after each.
        # maxpooling after the activations of the 2nd, 3rd, and 4th conv layers
        # 2 dense layers for classification
        # log_softmax
        #
        # As for the number of channels of each layers, try to experiment!

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 8,kernel_size = 3, padding =1),
            nn.ReLU(),
            
            nn.Conv2d(8, 32, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
            
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
            
            nn.Conv2d(64, 128, 3, 1), # 128  should be 12 !! --> output of layer can be output of model
            nn.ReLU(),
            nn.MaxPool2d(kernel_size =  2) 
        )
        
        # PAOLO DIDNT DO IT HERE AT THE BOTTOM
        
        # in_features of the first layer should be the product of the output shape of your feature extractor!
        # E.g. if the output of your feature extractor has size (batch x 128 x 4 x 4), in_features = 128*4*4=2048
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features = 2048, out_features= 2048), # powers of 2 (e.g., 2048) is better computationally
            nn.ReLU(),
            nn.Linear(in_features= 2048, out_features=10),  # the output should be 12
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        features = self.feature_extractor(x)
        print(features.shape) # batch, n_channels, height, weidth -> so in channels are n_channels * height * weight
        
        return self.classifier(features)

In [17]:
BATCH_SIZE = 35
LEARNING_RATE = 0.01
EPOCHS = 10
LOG_INTERVAL = 10

model = CNN().to(device)
# different optimiser so learning rate needs to be lower
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE) # optim.<OPTIMIZER_FLAVOUR>(model.parameters(), lr=LEARNING_RATE)


# no need to normalise mel form, output you normalised

train_loader, test_loader = (
    torch.utils.data.DataLoader(
        datasets.CIFAR10(DATA_PATH, train=train, transform=transform, download=True), # add my dataset
        batch_size=BATCH_SIZE,
        pin_memory=True,
        shuffle=train
    )
    for train in (True, False)
)

for epoch in range(EPOCHS):
    train(model, device, train_loader, optimizer, epoch, LOG_INTERVAL)
    test(model, device, test_loader)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
print(matrix.view(-1, 6))
print(matrix.view(-1,))
# print(matrix.view(-1, 3, -1)) # this doesn't work